In [2]:
import pandas as pd
import json

In [3]:
def filter_features(community_features, exclusion_set):
    return [feature for feature in community_features if feature not in exclusion_set]

In [10]:
file_path = 'Community_top10_features.json'

with open(file_path, 'r') as file:
    data = json.load(file)

In [11]:
exclusion_set = [
    "monde UPDATE", "suisse UPDATE", "monde ANALYSIS", "suisse ANALYSIS", 
    "economie UPDATE", "sciences-tech ANALYSIS", "sciences-tech UPDATE", 
    "economie ANALYSIS", "culture UPDATE", "sciences-tech EDUCATE", 
    "culture DIVERT", "monde EDUCATE", "suisse EDUCATE", "economie EDUCATE", 
    "suisse TREND", "culture ANALYSIS"
]

In [6]:
exclusion_set = []

In [11]:
filtered_communities = {community: filter_features(features, exclusion_set) for community, features in data.items()}

filtered_df = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in filtered_communities.items()]))

In [12]:
filtered_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,suisse INSPIRE,sciences-tech INSPIRE,culture INSPIRE,economie TREND,sciences-tech TREND,sciences-tech DIVERT,monde DIVERT,suisse DIVERT,culture EDUCATE,monde TREND,monde INSPIRE,culture TREND,monde DIVERT,culture INSPIRE,sport UPDATE,economie INSPIRE
1,culture INSPIRE,monde DIVERT,culture EDUCATE,monde DIVERT,monde DIVERT,monde DIVERT,monde TREND,monde DIVERT,monde DIVERT,monde DIVERT,monde DIVERT,monde DIVERT,monde TREND,monde DIVERT,culture INSPIRE,culture INSPIRE
2,monde DIVERT,monde TREND,monde DIVERT,monde TREND,monde TREND,monde TREND,culture INSPIRE,sciences-tech INSPIRE,monde TREND,culture INSPIRE,monde TREND,monde TREND,culture INSPIRE,monde TREND,suisse INSPIRE,monde DIVERT
3,sciences-tech INSPIRE,culture INSPIRE,monde TREND,culture INSPIRE,culture INSPIRE,sciences-tech INSPIRE,sciences-tech INSPIRE,culture EDUCATE,culture INSPIRE,culture EDUCATE,culture INSPIRE,culture EDUCATE,sciences-tech TREND,sciences-tech INSPIRE,culture EDUCATE,monde TREND
4,culture EDUCATE,culture EDUCATE,sciences-tech INSPIRE,culture EDUCATE,sciences-tech INSPIRE,culture INSPIRE,sciences-tech TREND,monde TREND,sciences-tech INSPIRE,sciences-tech TREND,sciences-tech INSPIRE,culture INSPIRE,suisse DIVERT,culture EDUCATE,monde TREND,sciences-tech INSPIRE
5,monde TREND,sciences-tech TREND,suisse DIVERT,sciences-tech TREND,culture EDUCATE,culture EDUCATE,culture EDUCATE,culture INSPIRE,suisse DIVERT,sciences-tech INSPIRE,culture EDUCATE,sciences-tech INSPIRE,culture EDUCATE,sciences-tech TREND,monde DIVERT,culture EDUCATE
6,suisse DIVERT,suisse DIVERT,sciences-tech TREND,sciences-tech INSPIRE,suisse DIVERT,sciences-tech TREND,suisse DIVERT,sciences-tech TREND,sciences-tech TREND,suisse DIVERT,sciences-tech TREND,suisse DIVERT,sciences-tech INSPIRE,suisse DIVERT,sport INSPIRE,suisse DIVERT
7,monde INSPIRE,suisse INSPIRE,suisse INSPIRE,suisse DIVERT,sciences-tech DIVERT,suisse DIVERT,economie TREND,suisse INSPIRE,monde INSPIRE,monde INSPIRE,suisse DIVERT,sciences-tech TREND,monde INSPIRE,sciences-tech DIVERT,sciences-tech TREND,sciences-tech DIVERT
8,sciences-tech TREND,monde INSPIRE,culture TREND,monde INSPIRE,monde INSPIRE,monde INSPIRE,sciences-tech DIVERT,monde INSPIRE,suisse INSPIRE,suisse INSPIRE,sciences-tech DIVERT,sciences-tech DIVERT,suisse INSPIRE,monde INSPIRE,monde INSPIRE,monde INSPIRE
9,economie TREND,economie TREND,economie TREND,sciences-tech DIVERT,economie TREND,suisse INSPIRE,monde INSPIRE,economie TREND,sciences-tech DIVERT,economie TREND,suisse INSPIRE,economie TREND,culture TREND,economie TREND,culture TREND,culture TREND


In [23]:
df5 = pd.DataFrame(dict([(k, pd.Series(v[:4])) for k, v in data.items()]))
df5_transposed = df5.transpose()

latex_table = df5_transposed.to_latex(header=False, index=True)

In [24]:
with open('community_features_table.tex', 'w') as tex_file:
    tex_file.write(latex_table)